In [2]:
import sys
sys.path.append('../src')
import os

import foolbox as fa
import model
import torch
import torchvision
from data import loader
from data import preprocess
from foolbox import PyTorchModel, accuracy
from skimage.metrics import peak_signal_noise_ratio as psnr  # 计算信噪比
from utils import common
from skimage.metrics import structural_similarity as ssim  # 计算图片平均结构相似度
import json

In [3]:
def load_dataset(path, is_train, is_shuffle, name, batch_size):
    _dataset = loader.Dataset(
        path, name=name, is_train=is_train,
        transform=torchvision.transforms.Compose([
            preprocess.CenterCrop(88), 
            torchvision.transforms.ToTensor()
        ])
    )
    data_loader = torch.utils.data.DataLoader(
        _dataset, batch_size=batch_size, shuffle=is_shuffle, num_workers=1
    )
    return data_loader

In [4]:
def evaluate(_m, ds):
    num_data = 0
    corrects = 0

    _m.net.eval()
    _softmax = torch.nn.Softmax(dim=1)
    for i, data in enumerate(ds):
        images, labels, _ = data
        predictions = _m.inference(images)
        predictions = _softmax(predictions)

        _, predictions = torch.max(predictions.data, 1)
        labels = labels.type(torch.LongTensor)
        num_data += labels.size(0)
        corrects += (predictions == labels.to(m.device)).sum().item()

    accuracy = 100 * corrects / num_data
    return accuracy

### 加载模型和数据集

In [36]:
config = common.load_config(os.path.join(common.project_root, 'experiments/config/AConvNet-SOC.json'))
model_name = config['model_name']


m = model.Model(
    classes=config['num_classes'], channels=config['channels'],
)

best_path = '/guoxuan/AConvNet-pytorch-main/experiments/model/AConvNet-SOC/model-030.pth'
test_set = load_dataset('dataset', False, True, 'soc', 100)

m.load(best_path)
m.net.eval()

load test data set: 0it [00:00, ?it/s]

load test data set: 2425it [00:01, 2376.13it/s]


Network(
  (_layer): Sequential(
    (0): Conv2DBlock(
      (_layer): Sequential(
        (conv): Conv2d(2, 16, kernel_size=(5, 5), stride=(1, 1), padding=valid)
        (relu): ReLU(inplace=True)
        (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): Conv2DBlock(
      (_layer): Sequential(
        (conv): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=valid)
        (relu): ReLU(inplace=True)
        (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (2): Conv2DBlock(
      (_layer): Sequential(
        (conv): Conv2d(32, 64, kernel_size=(6, 6), stride=(1, 1), padding=valid)
        (relu): ReLU(inplace=True)
        (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (3): Conv2DBlock(
      (_layer): Sequential(
        (conv): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=valid)
        (re

In [8]:
print(type(m.net), type(test_set))

<class 'model.network.Network'> <class 'torch.utils.data.dataloader.DataLoader'>


In [9]:
import torch
import torchvision
from torch.autograd import Variable
# from torch.autograd.gradcheck import zero_gradients
import torch.nn as nn
from torchvision import models
import numpy as np

In [10]:
def zero_gradients(x):
    if isinstance(x, torch.Tensor):
        if x.grad is not None:
            x.grad.detach_()
            x.grad.zero_()
    elif isinstance(x, collections.abc.Iterable):
        for elem in x:
            zero_gradients(elem)

In [11]:
# Saliency map
# 此方法为beta参数的简化版本，注重攻击目标贡献大的点
def saliency_map(F,x,t,mask):
    # F 为模型的输出
    # t 为攻击的类别
    # x 表示输入的图像
    # mask 标记位，记录已经访问的点的坐标
    F[0,t].backward(retain_graph=True)
    derivative=x.grad.data.cpu().numpy().copy()
    alphas=derivative*mask # 预测 对攻击目标的贡献
    betas=-np.ones_like(alphas) # 预测对非攻击目标的贡献
    sal_map=np.abs(alphas)*np.abs(betas)*np.sign(alphas*betas)
    idx=np.argmin(sal_map) # 最佳像素和扰动方向
    idx=np.unravel_index(idx,mask.shape) # 转换成(p1,p2)格式
    pix_sign=np.sign(alphas)[idx]
    return idx,pix_sign

In [12]:
for i, data in enumerate(test_set):
    images, labels, _ = data
    break

In [13]:
images.shape, labels.shape

(torch.Size([100, 2, 88, 88]), torch.Size([100]))

In [14]:
img = images[:1]

In [15]:
img.shape

torch.Size([1, 2, 88, 88])

In [16]:
img = img.to(m.device)

In [17]:
label = labels[0]

In [18]:
print(type(img), type(label))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [19]:
img.shape, label.shape

(torch.Size([1, 2, 88, 88]), torch.Size([]))

In [20]:
img.shape, img.device

(torch.Size([1, 2, 88, 88]), device(type='cuda', index=0))

In [21]:
label

tensor(0)

In [22]:
label.device

device(type='cpu')

In [23]:
net = m.net

In [24]:
device = m.device

In [25]:
img.requires_grad = True

In [26]:
epochs=500
theta=0.01 # 扰动系数
target_label=8 #攻击目标
target=Variable(torch.Tensor([float(target_label)]).to(device).long())
loss_func=torch.nn.CrossEntropyLoss()

mask=np.ones_like(img.data.cpu().numpy()) # 定义搜索域，修改后的位置置零
# 定义边界
max_=16.7701
min_=0
for epoch in range(epochs):
    output=net(img)
    label=np.argmax(output.data.cpu().numpy())
    print(target.shape, target, output.shape, output)
    loss=loss_func(output,target)
    print('epoch={} label={} loss={}'.format(epoch,label,loss))
    if label==target_label:
        break  # 攻击成功
    zero_gradients(img) # 梯度清零
    idx,pix_sign=saliency_map(output,img,target_label,mask)
    print("idx = ", idx, "pix_sign = ", pix_sign)
    # 添加扰动
    img.data[idx]=img.data[idx]+pix_sign*theta*(max_ - min_)
    # 达到极限的点不再参与更新
    if(img.data[idx] <= min_)or(img.data[idx] >= max_):
        print('idx={} over {}'.format(idx,img.data[idx]))
        mask[idx]=0
        img.data.cpu()[idx]=np.clip(img.data.cpu()[idx],min_,max_)

torch.Size([1]) tensor([8], device='cuda:0') torch.Size([1, 10]) tensor([[ 9.3760,  5.8173,  0.2222, -3.5017,  0.8242, -4.7103, -1.7983,  0.9111,
         -0.6978, -3.6384]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
epoch=0 label=0 loss=10.102455139160156
idx =  (0, 0, 22, 47) pix_sign =  1.0
torch.Size([1]) tensor([8], device='cuda:0') torch.Size([1, 10]) tensor([[ 9.2362,  5.6072,  0.3953, -3.6793,  0.5434, -4.4768, -1.8852,  0.7297,
         -0.1960, -3.5010]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
epoch=1 label=0 loss=9.45899772644043
idx =  (0, 0, 22, 47) pix_sign =  1.0
torch.Size([1]) tensor([8], device='cuda:0') torch.Size([1, 10]) tensor([[ 9.0884,  5.3819,  0.5744, -3.8725,  0.2508, -4.2229, -2.0042,  0.5473,
          0.3537, -3.3579]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
epoch=2 label=0 loss=8.75967788696289
idx =  (0, 0, 22, 47) pix_sign =  1.0
torch.Size([1]) tensor([8], device='cuda:0') torch.Size([1, 10]) tensor([[ 8.9452,  5.1620,  0.

In [27]:
img.shape, img.requires_grad

(torch.Size([1, 2, 88, 88]), True)

In [100]:
img.requries_grad = False

In [28]:
images.shape

torch.Size([100, 2, 88, 88])

In [29]:
ori_img = images[:1]

In [30]:
ori_img = ori_img.to(m.device)

In [31]:
img = img.detach()

In [32]:
ori_img.shape

torch.Size([1, 2, 88, 88])

## 计算指标

In [33]:
print(ori_img.shape, img.shape)

torch.Size([1, 2, 88, 88]) torch.Size([1, 2, 88, 88])


In [34]:
# 计算相关的指标
def calculate_avg_norm_distortion_factor(select_raw_images, select_adv_images, data_range=12):
    """
        计算相应的指标
    """

    # 没有对抗样本
    if select_raw_images.shape[0] == 0:
        return [0, 0, 0], 0, 0, [0, 0, 0]
        
    linf_avg_epsion = torch.norm(select_raw_images - select_adv_images, p=float('inf'), dim=(1, 2, 3))
    l1_avg_epsion = torch.norm(select_raw_images - select_adv_images, p=1, dim=(1, 2, 3))
    l2_avg_epsion = torch.norm(select_raw_images - select_adv_images, p=2, dim=(1, 2, 3))

    linf_distortion_factor = (linf_avg_epsion
                              / torch.norm(select_raw_images, p=float('inf'), dim=(1, 2, 3))).mean().item()
    l1_distortion_factor = (l1_avg_epsion
                            / torch.norm(select_raw_images, p=1, dim=(1, 2, 3))).mean().item()
    l2_distortion_factor = (l2_avg_epsion
                            / torch.norm(select_raw_images, p=2, dim=(1, 2, 3))).mean().item()

    psnr_item = psnr(select_raw_images.cpu().numpy(), select_adv_images.cpu().numpy(), data_range=data_range)

    #ssim_item = ssim(select_raw_images.permute(0, 2, 3, 1).cpu().numpy(), select_adv_images.permute(0, 2, 3, 1).cpu().numpy(),
                        # win_size=11, multichannel=True).item() # channel_last
    metric = dict()
    metric["l1_distortion_factor"] = l1_distortion_factor
    metric["l2_distortion_factor"] = l2_distortion_factor
    metric["linf_distortion_factor"] = linf_distortion_factor
    metric["psnr_item"] = psnr_item
    metric["l1_avg_epsion"] = l1_avg_epsion.mean().item()
    metric["l2_avg_epsion"] = l2_avg_epsion.mean().item()
    metric["linf_avg_epsion"] = linf_avg_epsion.mean().item()
    return metric

In [35]:
print(json.dumps(calculate_avg_norm_distortion_factor(ori_img, img), indent = 4))

{
    "l1_distortion_factor": 0.00015901018923614174,
    "l2_distortion_factor": 0.007133711129426956,
    "linf_distortion_factor": 0.3203635811805725,
    "psnr_item": 56.46433389246155,
    "l1_avg_epsion": 3.857123374938965,
    "l2_avg_epsion": 2.2436866760253906,
    "linf_avg_epsion": 2.0124123096466064
}


In [37]:
adv = img.data.cpu().numpy()[0]

In [38]:
print(adv.shape)

(2, 88, 88)


In [39]:
img.device

device(type='cuda', index=0)